In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import rdchiral.main as rdc
from tqdm import tqdm

from falsify_data import removemap
from utils import *

In [2]:
df = pd.read_csv('cleaned_uspto50k.csv')
rxn_smiles = list(df['rxn_smiles'])
retro_templates = list(df['retro_template'].drop_duplicates())
print(len(retro_templates))

13761


In [6]:
false_rxn_dict = {
    'rxn_smiles': [],
    'label': [],
    'true_rxn_smiles': [],
    'forward_template': []
}

for i in tqdm(range(len(df[:2]))):
# for i in [0]:
    rxn = rxn_smiles[i]
    rxn = removemap(rxn)
    rt, pt = rxn2rtpt(rxn)

    possible_pts = []
    for j in range(len(retro_templates)):
        # skip if the selected template is the one associated with the reaction
        # because this situation has already been taken into consideration
        # in the STRICT template mapping method
        if df.loc[i, 'retro_template'] == retro_templates[j]:
            continue
        rule = reverse_temp(retro_templates[j])
        possible_pts += rdc.rdchiralRunText(rule, rt)

    # remove duplicates from possible_pts (if any)
    possible_pts = list(set(possible_pts))

    # look for false reactions
    for possible_pt in possible_pts:
        # for possible product that is not the recorded product
        # we take it as a negative product
        # and the corresponding reaction as a negaitve sample
        if possible_pt != pt:
            false_rxn = f'{rt}>>{possible_pt}'

            # dump essential info into dict
            false_rxn_dict['rxn_smiles'].append(false_rxn)
            false_rxn_dict['label'].append(0)
            false_rxn_dict['true_rxn_smiles'].append(rxn)
            false_rxn_dict['forward_template'].append(rule)

100%|██████████| 2/2 [01:17<00:00, 38.67s/it]


In [7]:
false_rxn_df = pd.DataFrame(false_rxn_dict)
print(f'{len(false_rxn_df)} false reactions were succesfully generated this time,')

negative_random = pd.read_csv('negative_random.csv')
len_negative_random = len(negative_random)

negative_random = pd.concat([negative_random, false_rxn_df])
negative_random.drop_duplicates(subset='rxn_smiles', inplace=True)
negative_random.to_csv('negative_random.csv', index=False)
print(f'{len(negative_random) - len_negative_random} out of which are new.')
print(f'In total, {len(negative_random)} false reactions were successfully generated using random template mapping.')

32 false reactions were succesfully generated this time,
25 out of which are new.
In total, 105073 false reactions were successfully generated using random template mapping.
